In [1]:
import pandas as pd

file_path = r"C:\Users\charl\OneDrive\Documents\GitHub\5-CSRTT\data_performance\2025_12_12__17_50_08_659.csv"

# Skip the first 9 rows (metadata header), allow variable number of fields, handle empty cells
df = pd.read_csv(
    file_path,
    skiprows=9,
    engine='python',
    on_bad_lines='warn',
    keep_default_na=False,
    na_values=[''],
    header=0
)

# Rename columns
column_names = ["Num_line", "S", "MS", "Cat", "Num_cat", "State", "Display", "null"]
df.columns = column_names[:len(df.columns)]

df

C:\Users\charl\AppData\Local\Temp\ipykernel_12132\3597505872.py:6: ParserWarning: Skipping line 14: Expected 8 fields in line 14, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_12132\3597505872.py:6: ParserWarning: Skipping line 15: Expected 8 fields in line 15, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_12132\3597505872.py:6: ParserWarning: Skipping line 16: Expected 8 fields in line 16, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_12132\3597505872.py:6: ParserWarning: Skipping line 24: Expected 8 fields in line 24, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_12132\3597505872.py:6: ParserWarning: Skipping line 26: Expected 8 fields in line 26, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_12132\3597505872.py:6: ParserWarning: Skipping line 31: Expected 8 fields in line 31, saw 9

  df = pd.read_csv(
C:\Users\charl\AppData\Local\Temp\ipykernel_12132\35975058

,Num_line,S,MS,Cat,Num_cat,State,Display,null
0,1,0,0,Entry,0.0,Ready,NaN,NaN
1,2,0,5,Exit,0.0,Ready,Time,NaN
2,3,0,5,Entry,1.0,MagEntry,NaN,NaN
3,7,3,575,Input,1.0,On1A1,NaN,NaN
4,8,3,575,Exit,1.0,MagEntry,1,NaN
...,...,...,...,...,...,...,...,...
5001,7452,2398,760,Exit,1.0,MagEntry,1,NaN
5002,7453,2398,760,Entry,2.0,ITI2sec,NaN,NaN
5003,7454,2399,425,Input,33.0,Off1A1,NaN,NaN
5004,7455,2400,0,Exit,2.0,ITI2sec,Time,NaN


In [2]:
# Identify trials: each trial starts with "MagEntry" where Cat is "Entry" and ends with the next "MagEntry" with Cat "Entry"
# Find all indices where "MagEntry" appears in State column AND "Entry" in Cat column
magentry_indices = df[(df['State'] == 'MagEntry') & (df['Cat'] == 'Entry')].index.tolist()

# Create a list to store trial information
trials = []

# For each pair of consecutive MagEntry Entry events
for i in range(len(magentry_indices) - 1):
    trial_start_idx = magentry_indices[i]
    trial_end_idx = magentry_indices[i + 1]
    
    # Extract the trial data (from start to end, inclusive of end)
    trial_data = df.loc[trial_start_idx:trial_end_idx].copy()
    
    # Get all states/categories in this trial
    trial_states = trial_data['State'].tolist()
    
    # Store trial information using actual dataframe indices
    trials.append({
        'trial_number': i + 1,
        'start_line': trial_start_idx,
        'end_line': trial_end_idx,
        'start_idx': trial_start_idx,
        'end_idx': trial_end_idx,
        'states': trial_states,
        'num_events': len(trial_data)
    })

# Display trials summary
for trial in trials:
    print(f"Trial {trial['trial_number']}: Rows {trial['start_line']} to {trial['end_line']} ({trial['num_events']} events)")
    print(f"  States: {' -> '.join(trial['states'])}")
    print()

print(f"Total trials identified: {len(trials)}")

Trial 1: Rows 2 to 18 (17 events)
  States: MagEntry -> On1A1 -> MagEntry -> ITI2sec -> Off1A1 -> ITI2sec -> RandomHole -> RandomHole -> hole1 -> hole1 -> Holdhole1 -> Holdhole1 -> Omission -> Omission -> TimeOut -> TimeOut -> MagEntry

Trial 2: Rows 18 to 44 (27 events)
  States: MagEntry -> On1A1 -> MagEntry -> ITI2sec -> Off1A1 -> On1A1 -> ITI2sec -> RandomHole -> RandomHole -> hole3 -> Off1A1 -> hole3 -> Holdhole3 -> Holdhole3 -> Omission -> Omission -> TimeOut -> On1A4 -> Off1A4 -> On1A1 -> Off1A1 -> On1A1 -> Off1A1 -> On1A1 -> Off1A1 -> TimeOut -> MagEntry

Trial 3: Rows 44 to 68 (25 events)
  States: MagEntry -> On1A3 -> On1A4 -> Off1A3 -> Off1A4 -> On1A1 -> MagEntry -> ITI2sec -> Off1A1 -> On1A1 -> Off1A1 -> On1A1 -> ITI2sec -> RandomHole -> RandomHole -> hole4 -> Off1A1 -> hole4 -> Holdhole4 -> On1A2 -> On1A3 -> Holdhole4 -> CorrectResp -> CorrectResp -> MagEntry

Trial 4: Rows 68 to 96 (29 events)
  States: MagEntry -> Off1A2 -> Off1A3 -> On1A1 -> MagEntry -> ITI2sec -> Off1A